In [62]:
from langgraph.graph import StateGraph, START, END
from typing import Annotated, TypedDict, Sequence
from langchain_core.messages import (
    BaseMessage, SystemMessage, HumanMessage, AIMessage, ToolMessage
)
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
import fitz  # PyMuPDF
import os
from docx import Document  # For DOCX


In [ ]:

api_key="" #Provide your OpenAI API key here
resume_content=""
job_description=""

In [64]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages()]

def chunk_text(text, max_chunk_length=2000):
    return [text[i:i+max_chunk_length] for i in range(0, len(text), max_chunk_length)]

In [ ]:
# %%

@tool
def read_resume(file_path: str) -> str:
    """
    Reads the resume content from the specified file path.
    Acceptable file formats: .txt, .pdf, .docx
    """
    global resume_content
    if not file_path:
        return "File path not provided"
    
    if not os.path.exists(file_path):
        return "File Not Found"

    ext = os.path.splitext(file_path)[1].lower()
    
    try:
        if ext == ".txt":
            with open(file_path, "r", encoding="utf-8") as f:
                file_content = f.read()
        elif ext == ".pdf":
            doc = fitz.open(file_path)
            file_content = "\n".join([page.get_text() for page in doc])
            doc.close()
        elif ext == ".docx":
            from docx import Document
            doc = Document(file_path)
            file_content = "\n".join([para.text for para in doc.paragraphs])
        else:
            return "Unsupported file format"

        file_content = file_content.strip()
        if not file_content:
            return "File is empty"

        resume_content = file_content
        return "Resume successfully loaded."
    
    except Exception as e:
        return f"Error reading file: {str(e)}"


In [66]:
@tool
def get_job_description(description: str) -> str:
    """
    Accepts and stores the job description.
    """
    global job_description
    if not description.strip():
        return "Job description not provided"
    job_description = description.strip()
    return "Job description successfully received."

In [67]:
tools = [read_resume, get_job_description]
model = ChatOpenAI(model="gpt-4o", openai_api_key=api_key, temperature=0.1).bind_tools(tools)

In [68]:
def agent(state: AgentState) -> AgentState:
    """
    Main agent logic.
    """
    global resume_content, job_description

    system_prompt = SystemMessage(
        content=(
            "You are a helpful AI assistant that reads resumes and extracts relevant information.\n"
            "- Return error messages for: file not found, unsupported format, empty file, missing job description.\n"
            "- If both resume and JD are available, compare and suggest missing skills/courses required.\n"
            "- If experience is less than required in JD, return: 'Experience is less than required'.\n"
        )
    )

    messages = list(state["messages"])

    # If resume and job description are both present, chunk and feed
    if resume_content and job_description:
        for chunk in chunk_text(resume_content):
            messages.append(HumanMessage(content=f"Resume Chunk:\n{chunk}"))

        for chunk in chunk_text(job_description):
            messages.append(HumanMessage(content=f"Job Description Chunk:\n{chunk}"))

        user_message = HumanMessage(content="Analyze the resume against the job description and suggest missing skills or courses.")
        messages.append(user_message)

    response = model.invoke([system_prompt] + messages)

    print(f"\n🤖 AI: {response.content}")
    return {"messages": messages + [response]}

# %%


In [69]:
def should_continue(state: AgentState):
    """
    Determines whether to continue based on the state.
    Stops immediately on critical tool errors (like missing resume).
    """
    messages = state["messages"]
    FATAL_ERRORS = {
    "File Not Found",
    "Unsupported file format",
    "File is empty"
}

    if not messages:
        return "continue"

    # Check for fatal tool errors
    for msg in reversed(messages):
        if isinstance(msg, ToolMessage):
            if msg.content.strip() in FATAL_ERRORS:
                print(f"❌ Stopping due to critical error: {msg.content.strip()}")
                return "end"

    # Check if AI has already given final output
    for msg in reversed(messages):
        if isinstance(msg, AIMessage):
            if any(word in msg.content.lower() for word in ["missing skills", "courses", "skills you need"]):
                return "end"

    return "continue"


In [70]:
def print_messages(messages):
    """Function I made to print the messages in a more readable format"""
    if not messages:
        return
    
    for message in messages[-3:]:
        if isinstance(message, ToolMessage):
            print(f"\n🛠️ TOOL RESULT: {message.content}")

In [71]:
graph=StateGraph(AgentState)

graph.add_node("resume_agent",agent)
graph.add_node("tools",ToolNode(tools))
graph.set_entry_point("resume_agent")
graph.add_edge("resume_agent", "tools")

graph.add_conditional_edges("tools",should_continue,{
    "continue": "resume_agent",
    "end": END
})

app=graph.compile()


In [72]:
def run_resume_agent():
    print("\n ===== RESUME AGENT =====")
    
    state = {"messages": []}
    
    for step in app.stream(state, stream_mode="values"):
        if "messages" in step:
            print_messages(step["messages"])
    
    print("\n ===== FINISHED =====")

In [ ]:
if __name__ == "__main__":
    # Provide input here manually
    # First, load resume via tool
    resume_path = "" # provide full path to your resume file
    print(read_resume(resume_path))

    # Then, add JD
    jd_text = """
            
              """ #Provide the job description text here
    print(get_job_description(jd_text))

    # Finally run
    run_resume_agent()

Resume successfully loaded.
Job description successfully received.

 ===== RESUME AGENT =====

🤖 AI: Based on the provided resume and job description, here's an analysis of the missing skills or courses:

### Missing Skills/Courses:
1. **Containerization and Orchestration:**
   - The job description mentions the need for understanding containerized applications like Docker and container services like Kubernetes or ECS. The resume does not mention experience or knowledge in these areas.

2. **Document-Based Databases or Search Engines:**
   - Experience with document-based databases or search engines like Elasticsearch or MongoDB is mentioned in the job description but not in the resume.

3. **AWS Experience:**
   - The job description states that experience with AWS is a great plus. The resume lists Azure as a platform but does not mention AWS.

### Skills Present in Resume but Not Explicitly Required in JD:
- The resume lists experience with Azure, which is not mentioned in the job de